In [8]:
import tensorflow as tf
import pandas as pd
import cv2
import numpy as np
import os

# Ensure the image directory path is set correctly
image_dir = 'data/0'  # Use forward slash for compatibility

def load_data(annotation_path, image_dir):
    annotations = pd.read_csv(annotation_path)
    images, boxes, labels = [], [], []
    
    for idx, row in annotations.iterrows():
        img_path = os.path.join(image_dir, f'{row["filename"]}')  # Image path constructed correctly
        
        image = cv2.imread(img_path)
        
        # Check if image was loaded successfully
        if image is None:
            print(f"Warning: Image at path {img_path} not found or could not be loaded.")
            continue

        # Convert image to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224))  # Resize image to 224x224
        images.append(image)
        
        # Normalize bounding boxes by image dimensions
        h, w, _ = image.shape
        xmin = row['x'] / w  # Assuming 'x' and 'y' are the bounding box coordinates
        ymin = row['y'] / h
        xmax = row['w'] / w
        ymax = row['h'] / h
        boxes.append([xmin, ymin, xmax, ymax])
        
        # Convert class labels to numerical encoding if necessary
        labels.append(row['class'])  # Assuming 'class' is the label column
        
    return np.array(images), np.array(boxes), np.array(labels)

# Load data with the updated path
images, boxes, labels = load_data('data/class_0_annotations.csv', image_dir)

KeyError: 'class'

In [4]:
print("Images shape:", images.shape)  # Expected: (num_samples, 224, 224, 3)
print("Boxes shape:", boxes.shape)    # Expected: (num_samples, 4)


Images shape: (0,)
Boxes shape: (0,)


In [3]:
# Define the model (for simplicity, using a basic CNN model here)
def create_model():
    base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False)
    base_model.trainable = False
    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(4, activation='sigmoid')  # 4 outputs for bounding box coordinates
    ])
    return model

model = create_model()

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(images, boxes, epochs=10, batch_size=8)


Epoch 1/10


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(8,), dtype=float32). Expected shape (None, 224, 224, 3), but input has incompatible shape (8,)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(8,), dtype=float32)
  • training=True
  • mask=None